In [2]:
import sys 

class BestCut:
    def __init__(self, length, price, cuts):
        '''
        记录给定长度的最优切割方案，length对应长度，price对应最优价格，cuts对应切割方式
        如果一个8单位长的钢管最优切割方案是切成3单位和5单位，那么length=8,price=18,
        cuts记录3单位钢管的BestCut对象和5单位钢管的BestCut对象
                '''
        self.rodLength = length 
        self.bestPrice = price
        self.subBestCuts = cuts 
        
    def printCut(self):
        print("length: {0} , price: {1} ".format(self.rodLength, self.bestPrice))

In [4]:
def recursiveCutRod(priceTable, length, bestCutMap):
    if bestCutMap.get(length, None) is not None:
        #当前给定长度的最优切割方案已经找到
        return bestCutMap[length]
    q = None 
    if length == 0:
        q = BestCut(0, 0, None)
    elif length == 1:
        #单位长度不能继续往下切割
        return BestCut(1, priceTable[1], None)
    else:
        currentPrice = -sys.maxsize
        bestCut1 = None 
        bestCut2 = None
        for i in range(1, length):
            #把长度为length的钢管切割成i和length - i，然后递归的查找两部分的最优切割方案
            cut1 = recursiveCutRod(priceTable, i, bestCutMap)
            cut2 = recursiveCutRod(priceTable, length - i, bestCutMap)
            if cut1.bestPrice + cut2.bestPrice > currentPrice:
                currentPrice = cut1.bestPrice + cut2.bestPrice
                bestCut1 = cut1
                bestCut2 = cut2 
        #如果长度在价格表内，看看整体出售是不是更好
        if length < len(priceTable) and priceTable[length] > currentPrice:
            bestCutMap[length] = BestCut(length, priceTable[length], None)
            return bestCutMap[length]
        
        bestCutArray = [bestCut1, bestCut2]
        bestCutMap[length] = BestCut(length, currentPrice, bestCutArray)
        #积累
        return bestCutMap[length]
            
        
def cutRod(priceTable, length):
    bestCutMap = {}
    return recursiveCutRod(priceTable, length, bestCutMap)

def getBestCuts(cut, cutList):
    #找出切割的最优方案，当一个BestCut对象，其subBestCuts为空时，它本身就是最优切割方案中的一部分
    if cut.subBestCuts is None:
        cutList.append(cut)
        return 
    #如果subBestCut不为空，递归的从两个子切割中查找最优切割
    for subCut in cut.subBestCuts:
        getBestCuts(subCut, cutList)

In [5]:
priceTable = [0, 1, 5, 8, 9, 10, 17, 17, 20, 24, 30]
length = 15
cut = cutRod(priceTable, length)
print("the best price for rod with length {0} is {1}".format(length , cut.bestPrice))  

cutList = []
getBestCuts(cut, cutList) 
print("The best way to cut the rod with length of {0} are:".format(length))
for cut in cutList:
    cut.printCut()

the best price for rod with length 15 is 43
The best way to cut the rod with length of 15 are:
length: 2 , price: 5 
length: 3 , price: 8 
length: 10 , price: 30 


In [6]:
UP = 0
LEFT = 1
UP_LEFT = 2
#数组C用于记录最大共同子串长度，C[i][j]记录长度为i的X序列和长度为j的Y序列的最大共同子串长度
C = []
#数组B用于记录相关信息，它会用于构造最大共同子串
B = []

In [7]:
def getLongestCommonStringLength(X, Y):
    m = len(X)
    n = len(Y)
    #这里加1是因为算法描述中字符的下标从1开始，而代码对字符的访问，下标从0开始
    for i in range(0, m+1):
        C.append([])
        B.append([])
        for j in range(0, n+1):
            C[i].append(0)
            B[i].append(UP)
    for i in range (1, m+1):
        for j in range(1, n+1):
            #见1是因为字符下标从0开始
            if X[i-1] == Y[j-1]:
              #如果两字符串最后一个字符相同，那么最大共同子串就是X(i-1)与Y(j-1)的最大共同子串加上最后一个字符
                C[i][j] = C[i-1][j-1] + 1
                B[i][j] = UP_LEFT
            elif C[i-1][j] >= C[i][j-1]:
                #如果最后一个字符不同，那么最大共同子串就是X(i-1)与Y(j)的最大共同子串与X(i)和Y(j-1)的最大共同子串，两者长度最大那个
                C[i][j] = C[i-1][j]
                B[i][j] = UP 
            else:
                C[i][j] = C[i][j-1]
                B[i][j] = LEFT 
                
#根据数组C和B获取X(i)和Y(j)的最大共同子串
def  getLogestCommonString(X, Y, i, j):
    if i == 0 or j == 0:
        return ""
    lcs = ""
    if B[i][j] == UP_LEFT:
        #这里表明字符X[i]和Y[j]相同，因此最大共同子串应该包含该字符
        lcs = getLogestCommonString(X, Y, i-1, j-1)
        #字符的下标从0开始，所以要减1
        lcs += X[i-1]
    elif B[i][j] == UP:
        #这里表明X(i-1)和(j)的最大共同子串是X(i)和Y(j)的最大共同子串
        lcs = getLogestCommonString(X, Y, i-1, j)
    else:
        #这里表明X(i)和Y(j-1)的最大共同子串是X(i)和Y(j)的最大共同子串
        lcs = getLogestCommonString(X, Y, i, j-1)
    return lcs 

In [8]:
X="ABCBDAB"
Y="BDCABA"
getLongestCommonStringLength(X, Y)
lcs = getLogestCommonString(X, Y, len(X), len(Y))
print("longest common string of X and Y is {0}".format(lcs))

longest common string of X and Y is BCBA


In [9]:
def  modifiedGetLongestCommonSubStringLength(X, Y):
    m = len(X)
    n = len(Y)
    
    '''
    getLongestCommonStringLength在计算C[i][j]，的值时，只用到二维数组的上一行C[i-1][j-1]或C[i-1][j]的值，或者是
    同一行C[i][j-1]的值，因此数组C其实不需要n*m那么大，只需要2*m那么大就可以了，当前函数的逻辑与getLongestCommonStringLength
    一样，只不过把C的内存大小从n*m改为2*m
    '''
    C=[ [], []]
    for j in range(n+1):
        C[1].append(0)
        
    for i in range(1, m+1):
        C[0] = []
        #把第二行数据复制到第一行
        for j in range(len(C[1])):
            C[0].append(C[1][j])
            
        for j in range(1, n+1):
            if X[i-1] == Y[j-1]:
                C[1][j] = C[0][j-1] + 1
            elif C[0][j] > C[1][j-1]:
                C[1][j] = C[0][j]
            else:
                C[1][j] = C[1][j-1]
                
    return C[1]

In [10]:
def getLongestCommonSubStringWithLinearSpace(X,Y):
    if len(Y) == 0:
        return ""
    if len(X) == 1:
        for j in range(len(Y)):
            if X[0] == Y[j]:
                return X[0]
        return ""
    m = len(X)
    n = len(Y)
    i = int(m/2)
    #找到j，使得L=max(C(i,j) + C*(i,j))
    L1 = modifiedGetLongestCommonSubStringLength(X[0:i], Y)
    X1 = X[::-1]
    Y1 = Y[::-1]
    L2 = modifiedGetLongestCommonSubStringLength(X1[0: m - i], Y1)
    L = 0
    k = 0
    for j in range(0, n+1):
        if L1[j] + L2[n-j] > L:
            L = L1[j] + L2[n-j]
            k = j 
    '''
    找到满足条件的j后，我们再获取X(0,i)和Y(0,j)的最大共同子串S1,以及X(i+1, n) 和Y(j+1,n)的最大共同子串
    两者合在一起S1+S2就是X和Y的最大共同子串
    '''
    print("for string {0} and {1}".format(X, Y))
    print("seperate Y by index ", k)
    print("The longest sub string is combined by the longest sub string of {0} and {1} and sub string of {2} and {3}".format(X[0:i], Y[0:k],X[i:m], Y[k:n]))
    
    S1 = getLongestCommonSubStringWithLinearSpace(X[0:i], Y[0:k])
  
    S2 = getLongestCommonSubStringWithLinearSpace(X[i:m], Y[k:n])
  
    return S1+S2 

X="ABCBDAB"
Y="BDCABA"

L = getLongestCommonSubStringWithLinearSpace(X,Y)

print("The longest sub string of {0} and {1} is {2}".format(X,Y,L))


for string ABCBDAB and BDCABA
seperate Y by index  0
The longest sub string is combined by the longest sub string of ABC and  and sub string of BDAB and BDCABA
for string BDAB and BDCABA
seperate Y by index  2
The longest sub string is combined by the longest sub string of BD and BD and sub string of AB and CABA
for string BD and BD
seperate Y by index  1
The longest sub string is combined by the longest sub string of B and B and sub string of D and D
for string AB and CABA
seperate Y by index  2
The longest sub string is combined by the longest sub string of A and CA and sub string of B and BA
The longest sub string of ABCBDAB and BDCABA is BDAB
